In [1]:
from WS_Mdl.utils import Sign, Pre_Sign
import WS_Mdl.utils as U
import WS_Mdl.utils_imod as UIM
import WS_Mdl.geo as G
import os
from os import listdir as LD, makedirs as MDs
from os.path import join as PJ, basename as PBN, dirname as PDN, exists as PE
import shutil as sh
import re
from datetime import datetime as DT
from typing import Optional, Dict
import rasterio
from rasterio.transform import from_bounds
import imod
import numpy as np
import xarray as xr
from pathlib import Path
import pandas as pd
pd.set_option('display.max_rows', 30)      # show up to 30 rows
import geopandas as gpd
from shapely.geometry import Point as Pt
from concurrent.futures import ProcessPoolExecutor as PPE
import zipfile as ZF
import xml.etree.ElementTree as ET
import sys
from itertools import product
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import importlib as IL
IL.reload(U)
IL.reload(G)

<module 'WS_Mdl.geo' from 'c:\\users\\karam014\\onedrive - universiteit utrecht\\ws_mdl\\code\\WS_Mdl\\geo.py'>

# 1. Options.

In [3]:
#JypNote_name = 'write_SFR_NBr17.ipynb'  # replace with __file__ when converted to .py
#Pa_Ctmt = r'C:\OD\WS_Mdl\models\NBr\PoP\common\Pgn\Chaamse_beek\catchment_chaamsebeek_ulvenhout.shp'
Pa_TOP = r'C:\OD\WS_Mdl\models\NBr\In\TOP'
Pa_BOT = r'C:\OD\WS_Mdl\models\NBr\In\BOT'

In [4]:
MdlN, MdlN_B, MdlN_B_RIV = 'NBr19', 'NBr17', 'NBr1'

In [6]:
d_Pa = U.get_MdlN_paths(MdlN)
#d_Pa_B = U.get_MdlN_paths(MdlN_B)
Pa_SFR = PJ(d_Pa['Pa_Mdl'], f'In/SFR/{MdlN}/{MdlN}.SFR6')
MDs(PDN(Pa_SFR), exist_ok=True)  # Make dir for SFR file.

In [ ]:
Xmin, Ymin, Xmax, Ymax, cellsize, N_R, N_C = U.Mdl_Dmns_from_INI(d_Pa_B['INI'])

# 2. Load RIV to get params.

In [ ]:
DF_full = UIM.PRJ_to_DF(MdlN_B)

In [ ]:
DF = DF_full.copy().query(" (package == 'RIV') & (path==path) ")

# 3. Make an A of cells where we'll place SFR reaches.

## 3.0. Make a Series of As that show where each RIV IDF is active.

In [ ]:
def active_A(Pa):
    IDF = imod.formats.idf.open(Pa)
    IDF = IDF.sel(x=slice(Xmin, Xmax), y=slice(Ymax, Ymin))
    return IDF.notnull().astype(int)

In [ ]:
Se_active_A = DF['path'].apply(active_A)

## 3.1. Review the number of active cells

In [ ]:
DF['shape'] = Se_active_A.apply(lambda x: x.shape)
DF['N_active_cells'] = Se_active_A.apply(lambda x: int(x.sum()))
DF['basename'] = DF['path'].apply(lambda x: PBN(x))
DF['system'] = DF['basename'].apply(lambda x: x.split('_')[2].lower())

In [ ]:
l_Cs = ['parameter', 'time', 'shape', 'N_active_cells', 'system', 'basename']

In [ ]:
for time in DF['time'].unique():
    print(DF.query('time == "summer"').loc[:, l_Cs].sort_values(['N_active_cells', 'parameter'], ascending=False).to_string(index=False))
    print('-'*100)

There are some dataset issues:
- N_active_cells differ between parts of the same system
- N_active_cells same across seasons, so summer file can be used for all.

Notes:
- Sys1 is detailwatergangen
- Sys2 and Sys3 main river
- Sys4 and Sys5 not important here
- Each parameter has 15 arrays (3 seasons x 5 systems)

Check how many cells are active in RIV/DRN but not in detailwatergangen (cond).

In [ ]:
DF.query("basename == 'RIV_Cond_DRN_NBr1.IDF'")

Call them A1 and A2 for simplicity.

In [ ]:
A1 = Se_active_A.iloc[0]
A2 = Se_active_A.iloc[1]

In [ ]:
A3 = xr.where((A1==0) & (A2==1), 1, 0)
A4 = xr.where((A1==1) & (A2==1), 1, 0)
A5 = xr.where((A1==1) & (A2==0), 1, 0)

In [ ]:
A3.plot()

In [ ]:
A4.plot()

In [ ]:
A5.plot()

In [ ]:
A3.sum().values, A4.sum().values, A5.sum().values

- Most main river cells aren't part of detailwatergangen
- Detailwatergangen not well connected, main river cells help connectivity
- For NBr17 all cells will be connected downstream
- Will create one SFR network for all nodes within catchment for routing simplicity

Better to use cells active in both cond and bot arrays, maybe same as bot array only. Check next.

In [ ]:
Se_active_A.iloc[30:33].index.values, Se_active_A.iloc[0:3].index.values

In [ ]:
A6 = xr.where((Se_active_A.iloc[31-1]==1) & (Se_active_A.iloc[1-1]==0), 1, 0)
A7 = xr.where((Se_active_A.iloc[32-1]==1) & (Se_active_A.iloc[2-1]==0), 1, 0)

In [ ]:
print(f"""Number of active cells in the bot array that aren't active in the cond array.
            For detailwatergangen: {A6.sum().values}
            For RIV: {A7.sum().values}""")

Both 0, so bot array covers all bottom cells; use bot array.

Check opposite direction

In [ ]:
A8 = xr.where((Se_active_A.iloc[31-1]==0) & (Se_active_A.iloc[1-1]==1), 1, 0)
A9 = xr.where((Se_active_A.iloc[32-1]==0) & (Se_active_A.iloc[2-1]==1), 1, 0)

In [ ]:
print(f"""Number of active cells in the cond array that aren't active in the bot array.
            For detailwatergangen: {A8.sum().values}
            For RIV: {A9.sum().values}""")

Confirms plot of N_active columns.

## 3.2. Prep to write SFR blocks.

### 3.2.0. NREACHES

In [ ]:
A = Se_active_A.iloc[30] | Se_active_A.iloc[31]

In [ ]:
A.plot()

Check for any null values

In [ ]:
n_nans = A.isnull().sum().compute().item()
if n_nans > 0:
    raise ValueError(f'There are {n_nans} null values in the active cells array.')

In [ ]:
NREACHES = A.sum().values
NREACHES

### 3.2.1. PACKAGEDATA

In [ ]:
L = 1  # Will be improved to find correct layer
Rs, Cs = np.nonzero(A.values)
Xs, Ys = A['x'].values[Cs], A['y'].values[Rs]
Pts = [Pt(x, y) for x, y in zip(Xs, Ys)]

In [ ]:
A_rtp1 = imod.formats.idf.open(r'C:/OD/WS_Mdl\models\NBr\In\RIV\RIV_Bot_RIVER_RIVER_BOTTOM_NBr1.IDF').sel(x=slice(Xmin, Xmax), y=slice(Ymax, Ymin))
A_rtp2 = imod.formats.idf.open(r'C:/OD/WS_Mdl\models\NBr\In\RIV\NBr13\RIV_Stg_Detailwatergangen_NBr13.IDF').sel(x=slice(Xmin, Xmax), y=slice(Ymax, Ymin))

In [ ]:
# G.DA_stats(A_tp1)

In [ ]:
A_rtp1.plot()

In [ ]:
A_rtp2.plot()

Main rivers take priority over detailwatergangen for rtp.

In [ ]:
type(A_rtp1)

In [ ]:
# rtp_1 = A_rtp1.values[Rs, Cs]
# rtp_2 = A_rtp2.values[Rs, Cs]

In [ ]:
A_rtp_active = xr.where(np.isnan(A_rtp2), A_rtp1, A_rtp2)  # Use RIV/DRN bottoms if possible, else detailwatergangen

Cell above didn't work due to elevation conflicts.

#### Get correct Ls for each reach

l_TOP = [PJ(Pa_TOP, i) for i in LD(Pa_TOP) if ('idf' in i) and ('.dvc' not in i.lower())]
l_BOT = [PJ(Pa_BOT, i) for i in LD(Pa_BOT) if ('idf' in i) and ('.dvc' not in i.lower())]
TOP_100 = imod.formats.idf.open(l_TOP, pattern="{name}_L{layer}_").sel(x=slice(Xmin, Xmax), y=slice(Ymax, Ymin))
BOT_100 = imod.formats.idf.open(l_BOT, pattern="{name}_L{layer}_").sel(x=slice(Xmin, Xmax), y=slice(Ymax, Ymin))
**All reaches connected to L1, L1 top used as rtp. Will improve later.**
TOP = TOP_100.interp(y=A_rtp_active['y'], x=A_rtp_active['x'], method='linear', kwargs={'fill_value': 'extrapolate'})
BOT = BOT_100.interp(y=A_rtp_active['y'], x=A_rtp_active['x'], method='linear', kwargs={'fill_value': 'extrapolate'})

In [ ]:
Pa_TOP = r"C:\OD\WS_Mdl\models\NBr\Sim\NBr16\GWF_1\MODELINPUT\DIS6\TOPM_L1.IDF"

In [ ]:
TOP = imod.formats.idf.open(Pa_TOP)

In [ ]:
A_rtp = TOP.where(~np.isnan(A_rtp_active))
Se_rtp = A_rtp.stack(cell=('y','x')).dropna('cell').to_pandas().squeeze()

In [ ]:
type(Se_rtp)

#### Get correct L from TOP

---

valid_mask = ~np.isnan(A_rtp)

A_rtp_3D = A_rtp.where(valid_mask).expand_dims({'layer': TOP['layer']}, axis=0)

Cdt = (TOP >= A_rtp_3D) & (A_rtp_3D >= BOT)  # Check if layer bounds match

print(f"{NREACHES - Cdt.values.sum()} cells didn't match")

Check for multiple matches (should be impossible)

matches_per_cell = Cdt.sum(dim='layer')
multiple_matches = (matches_per_cell > 1)
if multiple_matches.any():
    print("Warning: some cells have multiple matching layers.")
else:
    print("No cells have multiple matching layers.")

Check cells without a match

no_match = (Cdt.sum(dim='layer') == 0)
valid_no_match = no_match & (~np.isnan(A_rtp))
rows, cols = np.where(valid_no_match.values)

for i, (r, c) in enumerate(zip(rows[:20], cols[:20])):
    ar_val = A_rtp.values[r, c]
    bot_vals = BOT.values[:, r, c]
    top_vals = TOP.values[:, r, c]
    print(f"Cell ({r},{c}): A_rtp={ar_val}")
    for l in range(len(bot_vals)):
        print(f"{i}  Layer {l}: BOT={bot_vals[l]}, TOP={top_vals[l]}")

Multiple unmatched cells found.

---

#### Make DF from params and write block

In [ ]:
DF_PD = pd.DataFrame({
    'ifno': np.arange(1, NREACHES+1),
    'L': L,
    'R': Rs+1,
    'C': Cs+1,
    'rlen': 25,
    'rwid': 2,
    'rgrd': 0.001,
    'rtp': Se_rtp,
    'rbth': 0.001,
    'rhk': 0.0001,
    'man': 0.03,
    'ncon': 1,
    'ustrf': 1,
    'ndv': 0,
    'aux': ['r' + str(i) for i in np.arange(1, NREACHES+1)],
    'X': Xs,
    'Y': Ys
})

### 3.2.2 CONNECTION

In [ ]:
GDF_Ctmt = gpd.read_file(Pa_Ctmt)
Pgn_Ctmt = GDF_Ctmt.geometry.union_all()

In [ ]:
DF_PD['geometry'] = Pts

In [ ]:
inside_mask = DF_PD['geometry'].apply(Pgn_Ctmt.contains)
DF_PD['Ctmt'] = 0
DF_PD.loc[inside_mask, 'Ctmt'] = 1

In [ ]:
d_outlets = {}

In [ ]:
for i in DF_PD['Ctmt'].unique():
    subset = DF_PD[DF_PD['Ctmt'] == i]
    d_outlets[i] = subset[subset['rtp'] == subset['rtp'].min()]['ifno'].values[0]
d_outlets

In [ ]:
DF_PD['ic'] = DF_PD.apply(lambda r: -int(d_outlets[r['Ctmt']]) if r['ifno'] != d_outlets[r['Ctmt']] else np.nan, axis=1)

In [ ]:
DF_PD['ncon']

In [ ]:
mapping = { int(-k): v for k, v in DF_PD['ic'].value_counts().to_dict().items() }

In [ ]:
mask = DF_PD['ifno'].isin(mapping)

In [ ]:
DF_PD.loc[mask, 'ncon'] = DF_PD.loc[mask, 'ifno'] # replace the ncon values with ifno, so that they can then be raplaced by mapping. This is quite flawed, but works for now.

In [ ]:
DF_PD.loc[mask, 'ncon'] = DF_PD.loc[mask, 'ncon'].map(mapping)

2 missing values; others reference outlets as expected.

In [ ]:
DF_PD['ic'] = DF_PD['ic'].fillna(0).astype(int)

In [ ]:
#DF_PD_group = DF_PD.groupby('ic')['ifno'].apply(lambda s: ' '.join(s.astype(str))).rename('ic_outflow')

In [ ]:
#DF_PD_group.index = DF_PD_group.index*(-1)

In [ ]:
ifnos_to_replace = DF_PD.loc[DF_PD['ic'] == 0, 'ifno']

In [ ]:
lookup = {ifno: ' '.join(DF_PD.loc[DF_PD['ic'] == -ifno, 'ifno'].astype(str))
          for ifno in ifnos_to_replace}
DF_PD.loc[DF_PD['ic'] == 0, 'ic'] = DF_PD['ifno'].map(lookup)

In [ ]:
DF_PD['ic'] = DF_PD['ic'].apply(lambda x: ' '.join(map(str, x)) if hasattr(x, '__iter__') and not isinstance(x, str) else str(x))

In [ ]:
DF_Conn = DF_PD[['ifno', 'ic']]

In [ ]:
DF_PD['ncon']

In [ ]:
DF_Conn['ic'].apply(lambda x: len(x.split(' ')))

## 3.3. Write SFR blocks

### 3.3.0. OPTIONS & DIMENSIONS

In [ ]:
with open(Pa_SFR, 'w') as f:
    f.write(f"# File created by {JypNote_name} for {MdlN} on {DT.now()}\n")
    f.write('BEGIN OPTIONS\nEND OPTIONS\n\n')
    f.write('BEGIN DIMENSIONS\n')
    f.write(f'\tNREACHES {NREACHES}\n')
    f.write('END DIMENSIONS\n')

### 3.3.1. PACKAGEDATA

In [ ]:
with open(Pa_SFR, 'a') as f:
    f.write('\nBEGIN PACKAGEDATA\n')
    block = U.DF_to_MF_block(DF_PD[DF_PD.columns[:-3]], Min_width=4)
    f.write(block)
    f.write('END PACKAGEDATA\n')

### 3.3.2. CONNECTIONDATA

In [ ]:
with open(Pa_SFR, 'a', newline='') as f:
    f.write('\nBEGIN CONNECTIONDATA\n')
    f.write('#' + '\t'.join(DF_Conn.columns) + '\n')
    for _, row in DF_Conn.iterrows():
        f.write(f"{row['ifno']}\t{row['ic']}\n")
    f.write('END CONNECTIONDATA\n')

In [ ]:
with open(Pa_SFR, 'a', newline='') as f:
    f.write('\nBEGIN PERIOD 1\n')
    f.write('END PERIOD\n')

#### Testing how iMOD interpolates values.

In [ ]:
2 * (3.54 - 3.7710) == (3.7710 - 4.2330)

In [ ]:
TOP.isel(layer=0, x=57, y=0).values

In [ ]:
TOP_100.isel(layer=0, x=13, y=0).values, TOP_100.isel(layer=0, x=14, y=0).values

In [ ]:
C54_5 = BOT_100.isel(layer=0, x=int((54.5-2.5)/4), y=1-1).values
C58_5 = BOT_100.isel(layer=0, x=int((58.5-2.5)/4), y=1-1).values

In [ ]:
step = (C58_5-C54_5)/8

In [ ]:
for i in range(9):
    if i%2:
        print(f"C{int(54+(i+1)/2)} - {C54_5 + i*step}")

In [ ]:
BOT.isel(layer=0, x=)